# <おまけ(でも重要)> 面倒な組み合わせテストを自動化する

DevCloudでテストを行う際、様々な条件を試すためにパラメーターを変更しながら何度も同じようなテストを行いたくなる場面が多々あります。  
このようなときに手動でパラメーターを変更しながら繰り返しテストを行うと手間がかかるだけでなく、条件漏れなどが発生し効率がよくありません。  
そこで、Pythonスクリプトを使用して単純作業を自動化する手法の一例を紹介します。

ここでは例としてOpenVINO付属の`benchmark_app`を使い、`googlenet-v1`と`squeezenet1.1`のモデルを各種条件で実行する手順をご紹介します

## 1. 必要なファイルを準備する
`benchmark_app`をOpenVINOインストールディレクトリよりコピーします。また、`Model downloader`と`Model converter`を使用し、IR形式の`googlenet-v1`と`squeezenet1.1`モデルをダウンロードしてきます。

In [ ]:
!cp -r $INTEL_OPENVINO_DIR/deployment_tools/tools/benchmark_tool .
!python3 $INTEL_OPENVINO_DIR/deployment_tools/tools/model_downloader/downloader.py --name squeezenet1.1,googlenet-v1
!python3 $INTEL_OPENVINO_DIR/deployment_tools/tools/model_downloader/converter.py  --name squeezenet1.1,googlenet-v1 --precisions FP16

----
ここからはPythonコードとなります

## 2. 自動化のためのクラス、関数を定義する
ここでは自動でテストパラメーターの更新およびパラメーター文字列を生成するためのクラスと、ジョブ実行を簡素化するヘルパー関数を定義しています。  
これらのクラス、関数を活用することでテストスクリプトを簡単に記述できるようになります。
次のセルを実行することで自動テストクラスが登録されます（登録されるだけなので何の出力も出ません）
- `autoParam class` : 渡されたパラメーターリストから、パラメーターの更新(`update()`)、パラメーター文字列の生成(`get()`)を行う
- `waitForJobCompletion(jobNum)`: 指定のジョブが終了するのを待つ。待っている間ステータスを表示する。
- `runjob(nodeName, jobFile, params)` : 指定のジョブファイルを指定のノードで実行する。パラメーターとしてparamsを渡す
- `printLineOfInterest(logFile, matchString)` : ログファイルから、指定した文字列を含む行だけ表示する

In [ ]:
# Automated test library
# Intel Corporation
# Yasunori Shimura

#           0      1       2           3      4    5
# val:   prefix, type, initial,      start, end, step
# enum:  prefix, type, initial_idx, item1, item2, item3,...
#
# e.g. parameter_list = [ [ '-nireq', 'val', 0, 0, 5, 1 ], [ '-flag', 'enum', 0, 'true', 'false', 'none'], [ '-precision', 'val', 0.5, 0.3, 0.8, 0.1 ] ]

# usage example:
# a=autoParam(parameter_list)
# print(a.get())
# a.update()

class autoParam:
    def __init__(self, param_list):
        self.param_list = param_list

    def update(self):
        exit_flag=False
        for param in self.param_list:
            if param[1]=='val':
                param[2]+=param[5]
                if param[2]>param[4]:
                    param[2]=param[3]
                else:
                    exit_flag=True
            elif param[1]=='enum':
                param[2]+=1
                if param[2]>=len(param)-3:
                    param[2]=0
                else:
                    exit_flag=True
            if exit_flag:
                return False
        return True                 # reached to end of the update loop (==Performed all parameter combination)

    def get(self):
        ret=''
        for param in self.param_list:
            if param[1]=='val':
                tmp=' {} {}'.format(param[0], param[2])
            elif param[1]=='enum':
                tmp=' {} {}'.format(param[0], param[param[2]+3])
            ret+=tmp
        return ret

# -----------------------------

import time
def waitForJobCompletion(jobNumber):
    print('Waiting for job completion...', end='')
    running=True
    while running:
        time.sleep(1)
        running=False
        status_list=!qstat           # Check job status
        for status in status_list:
            if jobNumber in status:  # if job_num is found in the status list, the job is still running
                running = True
        print(status.split()[4], end='')
    print('...Job {} completed'.format(jobNumber))    

import subprocess
def runJob(node_name, jobFile, params):
    # submit a job
    cmd='qsub -l {} {} -F "{}"'.format(node_name, jobFile, params)
    print(cmd)
    job_id=subprocess.check_output(cmd, shell=True).decode().strip()
    job_num = job_id.split('.')[0]
    print('job_id=', job_id)

    # wait for the job to complete
    waitForJobCompletion(job_num)
    return job_num

def printLineOfInterest(logFile, matchString):
    for l in [ line.strip() for line in open(logFile).readlines() ]:
        if matchString in l:
            print(l)   

## 3. ジョブスクリプトファイルを生成する
実際に送信するジョブスクリプトファイルを`%%writefile`マジックコマンドを使用して生成します。  
`$*`を使用して渡されたパラメーター文字列を受け取れるようにするのがポイントです。

In [ ]:
%%writefile job.sh
root=~/devcloud-workshop-jp
cd $root
python3 benchmark_tool/benchmark_app.py $*


## 4. autoParamクラスの使い方を学ぶ
autoParamのコンストラクタに次のようなリストを渡すと、半自動的にテストオプション文字列を生成させることが可能となります。
- `autoParam.update()` : パラメータを更新する。パラメーターは並んでる順番に先のものから更新される。パラメーターの値が終値に到達した場合、その次のパラメーターの値が更新される（数字の桁上がりのように）。最後のパラメーターに到達すると`False`を、それ以外の場合は`True`を返します。
- `autoParam.get()` : 現在のパラメーター値を使ってパラメーター文字列を生成する。

### パラメーターリストの書式
リストの書式は下記のようになります。パラメータータイプとしては`val`か`enum`が指定可能です。  
`val`の場合、`start`から`end`まで`step`単位で数値を増加させます。実数も使用可能ですが、精度には注意してください。  
`enum`の場合、並んでいる`item`を順にパラメーターとして使用します。とびとびの数値を使う場合にも`enum`を使用することができます。  
どちらのタイプでも、初期値の指定が可能です。これにより、中断したテストの途中から再開させるような使い方も可能になります。  
~~~
val:   [ option_flag_string , 'val',  initial_val, start, end, step ]
enum:  [ option_flag_string , 'enum', initial_idx, item1, item2, item3,... ]
~~~

次のセルを実行して実際の動作を確認してください。

In [ ]:
param_list = [ [ '-niter', 'enum', 0, '10', '100' ],
               [ '-nireq', 'val',  1, 1, 4, 1 ],
               [ '-m',     'enum', 0, 'public/squeezenet1.1/FP16/squeezenet1.1.xml', 'public/googlenet-v1/FP16/googlenet-v1.xml' ]
             ]

param=autoParam(param_list)
status=False
while status==False:
    param_str = param.get()
    print(param_str)
    status=param.update()

## 5. 実際にテストを行ってみる
では、実際にDevCloud上でのテストを自動化した例を見てみましょう。
下記のように簡単なスクリプトで組み合わせテストを自動化できるようになるので大変便利です。

ここでは`printLineOfinterest()`関数を使用して、必要最低限の情報だけ表示させることで表示を見やすくしています。

実際に実行してテスト結果を確認してみてください。
パラメーターの与え方でパフォーマンスに差が出ることも確認して下さい。

In [ ]:
param_list = [
    [ '-niter', 'enum', 0, '10', '100' ],
    [ '-nireq', 'val',  1, 1, 4, 1 ],
    [ '-m',     'enum', 0, 'public/squeezenet1.1/FP16/squeezenet1.1.xml', 
                           'public/googlenet-v1/FP16/googlenet-v1.xml' ]
]

param=autoParam(param_list)
status=False
while status==False:
    param_str = param.get()
    job_num = runJob('nodes=1:skylake', 'job.sh', param_str)

    # Display the result from the log file
    log_file = 'job.sh.o'+job_num
    printLineOfInterest(log_file, 'Resources:')
    printLineOfInterest(log_file, 'Count:')
    printLineOfInterest(log_file, 'Duration:')
    printLineOfInterest(log_file, 'Latency:')
    printLineOfInterest(log_file, 'Throughput:')

    status=param.update()     # update option parameter list for next iteration
    print('-'*40)

print('All test completed')

## 6. クリーンナップ
たくさんテストを行うとたくさんログファイルが生成されます。必要ないファイルは消してしまいましょう。

In [ ]:
!rm job.sh.o*
!rm job.sh.e*

----
ここまででDevCloud上でのテストの自動化手法の一例を学びました。  
このようにテストの自動化を行うことで効率と再現性を上げ、間違いを減らすことが可能となります。  

ディープラーニングを使用した推論プログラムは多くのパラメーターがパフォーマンスに関わり、それらの設定いかんでは数倍のパフォーマンス差が出ることもよくあります。  
テストを自動化することで手動では試せないような多くの組み合わせを試し、最適設定を探り出すことが可能になります。

今回の例を参考に、ご自身のベンチマークでもプラットフォーム特性に合わせた最適設定を探してみてください。

<おわり>